# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

__Мягкий дедлайн: 24.10 23:59__   
__Жесткий дедлайн: 27.10 23:59__

### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в почте или поисковой строке. За дополнительные баллы полученную систему нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex) или аналогов. В этой домашке вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей: основной и бонусной. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по __одному__ баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код.

При сдаче зададания в anytask вам будет необходимо сдать весь код, а если вы возьметесь за бонусную часть, то еще отчет и видео с демонстрацией вашего UI. За основную часть можно получить до __10-ти__ баллов, а за бонусную – до __5-ти__ баллов.

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd

emails = pd.read_csv('/kaggle/input/emails/emails.csv')
len(emails)

517401

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (2 балла).__ Очистите корпус текстов по вашему усмотрению и объясните свой выбор. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст.

In [2]:
emails['message'][0]

"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "

буду чистить по последнему заголовку из метаданных nX-FileName:. надо проверить, что каждое наблюдение в датасете имеет такой заголовок

In [3]:
import re

text = "aakldslsdldsdls\nX-FileName: pallen (Non-Privileged).pst.\n\nSome Normal Text.\nlya lya\n\n"
result = re.sub(r'(?s).*?X-FileName:[^\n]*\.[\s]*(.*)', r'\1', text)
print(result) 

Some Normal Text.
lya lya




проверим, что регулярное выражение нормально отрабатавает на всех данных с датасетах

In [4]:
# your code here
import pandas as pd
import re

pattern = re.compile(r'(?s).*?X-FileName:[^\n]*\.[\s]*(.*)')

def process_texts_vect(texts):
    cleaned_texts = []
    is_non_empty = []
    
    for text in texts:
        match = pattern.search(text)
        if match:
            cleaned = match.group(1)
        else:
            cleaned = text
        cleaned = cleaned.split('\n', 1)[-1] if '\n' in cleaned else cleaned
        cleaned_texts.append(cleaned)
        is_non_empty.append(1 if cleaned.strip() != '' else 0)
    
    return cleaned_texts, is_non_empty

cleaned, non_empty = process_texts_vect(emails['message'].tolist())
emails['cleaned_message'] = cleaned
emails['is_non_empty'] = non_empty

non_empty_count = emails['is_non_empty'].sum()

In [5]:
print(non_empty_count)

517401


In [6]:
emails['cleaned_message'][3]

'\nRandy,\n\n Can you send me a schedule of the salary and level of everyone in the \nscheduling group.  Plus your thoughts on any changes that need to be made.  \n(Patti S for example)\n\nPhillip'

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат для финального пользователя будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы заметите, что из-за этого падает качество самого текса, то можете удалить все небуквенные символы на этапе токенизации.

In [8]:
import string
import re

def clean_text(text):
    text = re.sub(r'[^\w\s]|\d+', '', text)
    return text

emails['cleaned_message'] = emails['cleaned_message'].apply(clean_text)

In [9]:
emails['cleaned_message'] = emails['cleaned_message'].apply(clean_text)

In [10]:
emails['cleaned_message']

0                               \nHere is our forecast\n\n 
1         \nTraveling to have a business meeting takes t...
2                              \ntest successful  way to go
3         \nRandy\n\n Can you send me a schedule of the ...
4                            \nLets shoot for Tuesday at   
                                ...                        
517396    \nThis is a trade with OILSPECHEDGENG John Lav...
517397    \nSome of my position is with the Alberta Term...
517398    \n\n\n Original Message\nFrom \tDoucet Dawn  \...
517399    \nAnalyst\t\t\t\t\tRank\n\nStephane Brodeur\t\...
517400    \ni think the YMCA has a class that is for peo...
Name: cleaned_message, Length: 517401, dtype: object

In [11]:
emails['tokenized_message'] = emails['cleaned_message'].str.findall(r'\w+')

In [12]:
emails['tokenized_message']

0                                 [Here, is, our, forecast]
1         [Traveling, to, have, a, business, meeting, ta...
2                           [test, successful, way, to, go]
3         [Randy, Can, you, send, me, a, schedule, of, t...
4                           [Lets, shoot, for, Tuesday, at]
                                ...                        
517396    [This, is, a, trade, with, OILSPECHEDGENG, Joh...
517397    [Some, of, my, position, is, with, the, Albert...
517398    [Original, Message, From, Doucet, Dawn, Sent, ...
517399    [Analyst, Rank, Stephane, Brodeur, Chad, Clark...
517400    [i, think, the, YMCA, has, a, class, that, is,...
Name: tokenized_message, Length: 517401, dtype: object

## Общая схема решения

Мы хотим сделать систему, которая будет ускорять набор текста, советуя подходящие продолжения. Для подсказки следующего слова мы будем использовать n-граммную модель. Так как n-граммная модель работает с целыми словами, а советы мы хотим давать в риал-тайме даже когда слово еще не дописано, сперва надо научиться дополнять слово до целого.

## Дополнение слова

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова не больше чем за $O(n + mk)$, где $m$ - число подходящих слов, а $k$ – длина суффикса.

__Задание 2 (2 балла).__ Допишите префиксное дерево для поиска слов по префиксу.

In [13]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        for word in vocabulary:
            current = self.root
            for symbol in word:
                if symbol not in current.children:
                    current.children[symbol] = PrefixTreeNode()
                current = current.children[symbol]   
            current.is_end_of_word = True
        # your code here

    def search_prefix(self, prefix) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """
        words = []
        current = self.root
        for char in prefix:
            if char not in current.children:
                return words
            current = current.children[char]

        def _dfs(current, path):
            if current.is_end_of_word:
                words.append("".join(path))
                
            for c, children_node in current.children.items():
                _dfs(children_node, path + [c])
                
        _dfs(current, list(prefix))

        return words
                

        # your code here

In [14]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его __встречаемости в корпусе__.

__Задание 3 (2 балла).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [99]:
from itertools import chain
from collections import Counter
from typing import List

class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – корпус текстов
        """
        # your code here
        vocabulary = list(chain.from_iterable(corpus)) #из списка списком делаем один список
        all_words = len(vocabulary)
        count = Counter(vocabulary) #считаем количество попаданий для каждого слова в тексте
        self.words_probs = {key: value / all_words for key, value in count.items()} #считаем вероятности
        u_words = list(set(vocabulary)) #убираем повторяющиеся слова
        self.prefix_tree = PrefixTree(u_words)

    def get_words_and_probs(self, prefix: str) -> (List[str], List[float]):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words, probs = [], []
        # your code here
        words = self.prefix_tree.search_prefix(prefix)
        for word in words:
            if word in self.words_probs:
                probs.append(self.words_probs[word])
        return words, probs

In [100]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему следующее слово (или несколько) с учетом дописанного. Для этого мы воспользуемся n-граммной моделью.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

$P(w_i \mid w_{i-1}, \dots, w_{i-n})$ оценивается по частоте встречаемости n-граммы.   

__Задание 4 (2 балла).__ Напишите класс для n-граммной модели. Никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [1]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from itertools import chain
from nltk.util import ngrams

corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

list(chain.from_iterable(ngrams(seq, 2+1) for seq in corpus)) 

[('aa', 'aa', 'aa'),
 ('aa', 'aa', 'aa'),
 ('aa', 'aa', 'ab'),
 ('abb', 'aa', 'ab'),
 ('aa', 'ab', 'bba'),
 ('ab', 'bba', 'bbb'),
 ('bba', 'bbb', 'bcd')]

In [111]:
from nltk.util import ngrams
from collections import Counter
from itertools import chain
from fractions import Fraction


class NGramLanguageModel:
    def __init__(self, corpus, n):

        self.ngrams = list(chain.from_iterable(ngrams(seq, n+1) for seq in corpus))  
        self.n = n
        # your code here

    def get_next_words_and_probs(self, prefix: list) -> (List[str], List[float]):
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов
        """
        next_words, probs = [], []

        endings = []

        for ngram in self.ngrams:
            if " ".join(ngram[:self.n]) == " ". join(prefix):
                endings.append(ngram[self.n])

        probs_dict = Counter(endings)

        all_endings = len(endings)

        for key, value in probs_dict.items():
            next_words.append(key)
            probs.append(value / all_endings)
        # your code here

        return next_words, probs

In [112]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (2 балла).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. После этого можно добавить опцию генерации нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [113]:
from typing import Union

class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """

        suggestions = []

        for _ in range(n_texts):
            current_word = text[-1]
            context = text[:-1] if len(text) > 1 else []
            words, _ = self.word_completor.get_words_and_probs(current_word)
            
            recommended_word = words[0]
            current_suggestion = [recommended_word]
            
            current_context = context + [recommended_word]
            
            for i in range(n_words):
                
                prefix = current_context[-self.n_gram_model.n:] if len(current_context) >= self.n_gram_model.n else current_context
                next_words, probs = self.n_gram_model.get_next_words_and_probs(prefix)
                most_probable_word = next_words[0]
                
                current_suggestion.append(most_probable_word)
                current_context.append(most_probable_word)
                
            suggestions.append(current_suggestion)

        # your code here

        return suggestions

In [ ]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [450]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

## Бонусная часть: Добавляем UI

Запускать ячейки в юпитере – это хорошо, но будет лучше, если вашим решением действительно можно будет пользоваться. Для этого вам предлагается добавить полноценных User Interface. Мы рекомендуем использовать для этого [reflex](https://github.com/reflex-dev/reflex). Это Python библиотека для создания web-интерфейсом с очень богатым функционалом.

Ваша задача – сделать поле для текстового ввода, при наборе текста в котором будут появляться подсказки в реальном времени. Продумайте, как пользователь будет выбирать подсказки, сколько продолжений рекомендавать и так далее. В общем, должно получиться красиво и удобно. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным. 

За это задание можно получить до __5-ти бонусных баллов__ в зависимости о того, насколько хорошо и удобно у вас получилось. При сдаче задания прикрепите небольшой __отчет__ (полстраницы) с описанием вашей системы, а также __видео__ (1-2 минуты) с демонстрацией работы интерфейса.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.